In [12]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from datetime import datetime

from __future__ import print_function
import pickle

In [13]:
url = 'https://www.the-numbers.com/movie/budgets/all/201'

user_agent = {'User-agent': 'Mozilla/5.0'}

response  = requests.get(url, headers = user_agent)

page = response.text

soup = BeautifulSoup(page,"lxml")

In [14]:
table = soup.find('table')
table

<table>
<tr><th> </th><th>Release<br/>Date</th><th>Movie</th><th>Production<br/>Budget</th><th>Domestic<br/>Gross</th><th>Worldwide<br/>Gross</th></tr>
<tr><td class="data">201</td>
<td><a href="/box-office-chart/daily/2012/03/28">Mar 28, 2012</a></td>
<td><b><a href="/movie/Wrath-of-the-Titans#tab=summary">Wrath of the Titans</a></b></td>
<td class="data"> $150,000,000</td>
<td class="data"> $83,670,083</td>
<td class="data"> $305,270,083</td>
</tr>
<tr><td class="data">202</td>
<td><a href="/box-office-chart/daily/2007/11/16">Nov 16, 2007</a></td>
<td><b><a href="/movie/Beowulf#tab=summary">Beowulf</a></b></td>
<td class="data"> $150,000,000</td>
<td class="data"> $82,280,579</td>
<td class="data"> $195,080,579</td>
</tr>
<tr><td class="data">203</td>
<td><a href="/box-office-chart/daily/2012/05/09">May 9, 2012</a></td>
<td><b><a href="/movie/Dark-Shadows#tab=summary">Dark Shadows</a></b></td>
<td class="data"> $150,000,000</td>
<td class="data"> $79,727,149</td>
<td class="data"> $2

In [15]:
rows = [row for row in table.find_all('tr')]

In [16]:
movies = {}

for row in rows[1:]:
    items = row.find_all('td')
    link = row.find('a')
    title, url = link.text, link['href']
    movies[title] = [url] + [i.text for i in items]
    
movies

{'Mar 28, 2012': ['/box-office-chart/daily/2012/03/28',
  '201',
  'Mar 28, 2012',
  'Wrath of the Titans',
  '\xa0$150,000,000',
  '\xa0$83,670,083',
  '\xa0$305,270,083'],
 'Nov 16, 2007': ['/box-office-chart/daily/2007/11/16',
  '202',
  'Nov 16, 2007',
  'Beowulf',
  '\xa0$150,000,000',
  '\xa0$82,280,579',
  '\xa0$195,080,579'],
 'May 9, 2012': ['/box-office-chart/daily/2012/05/09',
  '203',
  'May 9, 2012',
  'Dark Shadows',
  '\xa0$150,000,000',
  '\xa0$79,727,149',
  '\xa0$238,202,668'],
 'Jun 28, 2013': ['/box-office-chart/daily/2013/06/28',
  '204',
  'Jun 28, 2013',
  'White House Down',
  '\xa0$150,000,000',
  '\xa0$73,103,784',
  '\xa0$205,440,387'],
 'Feb 12, 2010': ['/box-office-chart/daily/2010/02/12',
  '205',
  'Feb 12, 2010',
  'The Wolfman',
  '\xa0$150,000,000',
  '\xa0$62,189,884',
  '\xa0$142,634,358'],
 'Dec 15, 2016': ['/box-office-chart/daily/2016/12/15',
  '206',
  'Dec 15, 2016',
  'The Great Wall',
  '\xa0$150,000,000',
  '\xa0$45,157,105',
  '\xa0$334,486,

In [17]:
budget3 = pd.DataFrame(movies).T  #transpose
budget3.columns = ['link_stub','rank_all_movies','release','title', 
                    'budget', 'domestic_gross','worldwide_gross']

budget3.shape

(97, 7)

In [18]:
budget3['budget'] = budget3['budget'].str.replace('$', '')
budget3['budget'] = budget3['budget'].str.replace(',', '')
budget3['budget'] = budget3['budget'].astype(int)

In [19]:
budget3.head()

,link_stub,rank_all_movies,release,title,budget,domestic_gross,worldwide_gross
"Mar 28, 2012",/box-office-chart/daily/2012/03/28,201,"Mar 28, 2012",Wrath of the Titans,150000000,"$83,670,083","$305,270,083"
"Nov 16, 2007",/box-office-chart/daily/2007/11/16,202,"Nov 16, 2007",Beowulf,150000000,"$82,280,579","$195,080,579"
"May 9, 2012",/box-office-chart/daily/2012/05/09,203,"May 9, 2012",Dark Shadows,150000000,"$79,727,149","$238,202,668"
"Jun 28, 2013",/box-office-chart/daily/2013/06/28,204,"Jun 28, 2013",White House Down,150000000,"$73,103,784","$205,440,387"
"Feb 12, 2010",/box-office-chart/daily/2010/02/12,205,"Feb 12, 2010",The Wolfman,150000000,"$62,189,884","$142,634,358"


In [20]:
budget3.to_csv('budget3.csv')